In [ ]:
import pandas as pd
import folium

# Import the csv file with radiosonde data (replace the path and file name with your own)
df = pd.read_csv('/home/arthur/radiossonda/dados/brutos/jun23/RS-stm-gpx-2023-06-25-12Z.csv')

# Remove rows with NaN values in Latitude and Longitude columns
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Calculate wind speed at each point
#df['Speed'] = (df['Eastward Wind Speed']**2 + df['Northward Wind Speed']**2)**0.5

# Get the first latitude, longitude and altitude coordinates
lat, lon, alt, climb = df.iloc[0][['Latitude', 'Longitude', 'Altitude', 'Climb']]

# Create a map object centered on the first coordinates
mapa = folium.Map(location=[lat, lon], zoom_start=10)

# Create a list of point coordinates
coordinates = list(zip(df['Latitude'], df['Longitude'], df['Altitude'], df['Climb']))

def get_color(velocity):
    if velocity < 10:
        return 'green'
    elif velocity < 20:
        return 'yellow'
    else:
        return 'red'

# Add a line connecting all points on the map
line_segments = []
for i in range(1, len(coordinates)):
    start = coordinates[i-1][0:2]
    end = coordinates[i][0:2]
    climb = coordinates[i][3]
    velocity = df.iloc[i]['Speed']
    color = get_color(velocity)
    segment = {'start': start, 'end': end, 'color': color}
    line_segments.append(segment)
    folium.PolyLine(locations=[start, end], color=color).add_to(mapa)
    
    # Identify the first point where there was a drop in altitude
    if climb < 0:
        folium.Marker(location=end, icon=folium.Icon(color='black'),
                      popup='BALÃO ESTOUROU! Altitude: {:.2f} m'.format(coordinates[i][2])).add_to(mapa)
        break

# Save the map to an HTML file
mapa.save('trajetoria.html')

# Display the map
display(mapa)
